This script creates the SUBJECTSFILE and demographics.tsv which are used for the swarmplot. Also, it creates the .fsgd file, which determines which subjects should be included on the GLM as well as the symlinks which are set up in each pipelines respective analyses_nip directory. The recon_notes.csv in their respective pipelines determine who will be excluded from this list (including columns EXCLUDE=1.0 and STATUS_RECONEDIT=complete for flair and STATUS_INITIAL=complete for 3T).
???For all cases, only subjects of whom we have their AMY status are included in the analysis.?? 
Total of 46 jobs are submitted with this script:
Jobs: aparc for lh+rh (area, vol, thickness, thicknessstd) and aseg (vol, mean, std) for the 3T and FLAIR pipelines = total of 22 jobs
Jobs: glm group steps for lh+rh (mris_preproc, mri_surf2surf, and glm_fit) for the 2 conditions (AMYneg vs. CN and AMYpos vs. CN) for both the 3T and FLAIR pipelines = total of 24 jobs.

In [67]:
# TO DO: make generalizable to any project; make the sleep time dynamic to the number of processing elements.
import os
import pandas as pd
import pickle
import csv
import numpy as np
import glob
from os import system
from paramiko import SSHClient
import time

project = 'leads' # specifiy project (this is make for leads)

# make new stats file of the directory of subjects
# make new demographic file by using the 
pipelines = ['RECON_FLAIR','RECON_3T']
downloads_info = '/autofs/cluster/animal/scan_data/leads/spreadsheets/LONI_DOWNLOADS/combined_downloads.csv'
subjectinfo = pd.read_csv(downloads_info)
amyinfo = pd.read_csv('/autofs/cluster/animal/scan_data/leads/spreadsheets/LONI_PETINFO/LEADS_AMYELG.csv')
fsaverage_path = '/autofs/cluster/freesurfer/centos7_x86_64/stable6/subjects/fsaverage/'
scripts_path = '/autofs/cluster/animal/scan_data/leads/analyses_nip/scripts/'

In [68]:
def credentials(): # combined with find_dicom
    import getpass
    USER = getpass.getuser()
    print('Please enter your PASSWORD for launchpad access: ')
    PASS= getpass.getpass()
    return USER, PASS

[user,pw] = credentials()

host="launchpad"
client=SSHClient()
client.load_system_host_keys()
client.connect(host,username=user,password=pw, look_for_keys=False)

Please enter your PASSWORD for launchpad access: 
········


In [6]:
# loop for each pipeline (start with FLAIR)
# list all folder in directory that begin with LDS
group_array = [[],[]]
amy_array = [[],[]]
sub_array = []


for idx, pip in enumerate(pipelines): 
    if pip == 'RECON_FLAIR':
        recon = 'edit'
    elif pip == "RECON_3T":
        recon = 'FS'
    analysis_path = '/autofs/cluster/animal/scan_data/leads/analyses_nip/'+pip+'/'
    recon_path = '/autofs/cluster/animal/scan_data/leads/recon_nip/'+pip+'/'
    recon_notes = pd.read_csv(recon_path+'/recon_notes.csv')
    os.chdir(analysis_path) 
    # look for all subjects that are determined for inclusion and are complete in processing (keeping those who are flagged fow now)
    good_subjects = recon_notes[(recon_notes['EXCLUDE']== 0.0) & (recon_notes['STATUS_FINAL']== 'complete')]
    sub_array.append(good_subjects['LEADS_ID'].tolist())
        
    # now make a list of new subject IDs with there corresponding group and write to text file
    for sub in sub_array[idx]:
        print(sub)
        folders = []

        # if fsaverage is not in folder (make symlink)
        if not os.path.islink(analysis_path+'fsaverage'):
            os.symlink(fsaverage_path, analysis_path+'fsaverage')

        # copy over symlink if not there (only if INCLUDE and COMPLETE -- still could be left out of final analyses
        # folder if no AMY info)
        # check if symlink exists in analysis folder, 
        if os.path.exists(analysis_path+sub):
            pass
        else:
            try:
                folders = [x for x in os.listdir(recon_path+sub) if x.startswith(recon)] 
                editreconpath = recon_path+sub+'/'+folders[0]
                os.symlink(editreconpath, analysis_path+sub)
                print('linked '+editreconpath, analysis_path+sub)
            except(IndexError):
                print(sub+" has no edited recon yet.")
        
        #group_array[idx].append(subjectinfo[subjectinfo['Subject']==sub.split('_')[0]].Group.values[0])
        if subjectinfo[subjectinfo['Subject']==sub.split('_')[0]].Group.values[0] == 'CN':
            group_array[idx].append(subjectinfo[subjectinfo['Subject']==sub.split('_')[0]].Group.values[0])
            #print(sub+' is in group '+subjectinfo[subjectinfo['Subject']==sub.split('_')[0]].Group.values[0])
        elif subjectinfo[subjectinfo['Subject']==sub.split('_')[0]].Group.values[0] == 'PT':
            try:
                group_array[idx].append(str(amyinfo[amyinfo['subject_code']==sub.split('_')[0].lower()].outcome.values[0]))
                #print(sub+' is in group '+subjectinfo[subjectinfo['Subject']==sub.split('_')[0]].Group.values[0]+' and has AMY '+str(amyinfo[amyinfo['subject_code']==sub.split('_')[0].lower()].outcome.values[0]))
            except(IndexError):
                #print(sub+' is in group '+subjectinfo[subjectinfo['Subject']==sub.split('_')[0]].Group.values[0]+' and has AMY '+str(amyinfo[amyinfo['subject_code']==sub.split('_')[0].lower()].outcome.values[0]))
                group_array[idx].append("") # change this to PT if want without AMY status
        else:
            #print(sub+ ' has no group data.')
            group_array[idx].append("")

    # further refine group (replace all PT with AMY+ or AMY-)
    group_status = [w.replace('0', 'N').replace('1','P').replace('CN','C') for w in group_array[idx]] ## I THINK PROBLEM STARTS HERE

    
    # put into a dataframe
    zippedList =  list(zip(sub_array[idx], group_status)) # group_stats is redefined per iteration
    demographics_df = pd.DataFrame(zippedList, columns = ['Subject' , 'Group']) #, index=['a', 'b', 'c']) 
    demographics_df.set_index('Subject', inplace=True)
    
    # now create new dataframe eliminating instances of PT (ones that do not have AMY status)
    cleaned_demographics = demographics_df[demographics_df['Group']!='']
    #demographics_df = demographics_df[demographics_df.Group != '']
    
    # now save this to the appropriate analysis pipeline
    cleaned_demographics.to_csv(analysis_path+'/demographics_generated', sep='\t') #, index=False
    
    
    # NOTE:  where did Jess get those other subjects from?
    # ignore for now
    
    # create a SUBJECTSFILE WITH THIS LIST and save to respective analysis folder
    # (this will determine what stats will be generated)
        
    with open(analysis_path+'SUBJECTSFILE_automated', 'w') as f: # does not exclude subjects with no AMY status
        for item in cleaned_demographics.index.values.tolist():
            f.write("%s\n" % item)
            
    # now also make fsgd file for GLM for each pipeline (compare anything other than C to C)
    group_list = list(cleaned_demographics.Group.unique())
    group_list.remove('C')
    
    for inst in group_list:
        # make new data frame with only that inst and C, then add a column Input
        new_compare = cleaned_demographics.loc[cleaned_demographics['Group'].isin(['C',inst])]
        sublist = new_compare.index.values.tolist()
        sublist.sort()
        grouplist = list(new_compare['Group'])
        indexlist = ['Input'] * len(grouplist)

        with open(analysis_path+'/AMY'+inst+'vsCN.fsgd', 'w') as f:
            writer = csv.writer(f, delimiter='\t')
            f.write('GroupDescriptorFile 1\n# One Factor/Two Levels (no covariates)\nTitle LEADS\nClass C plus green\nClass %s circle blue\nVariables\n' %inst) #%inst
            writer.writerows(zip(indexlist,sublist,grouplist))
#         os.system("cat AMYPvsCN.fsgd | sed 's/\r/\n/g' > new.AMYPvsCN.fsgd")  # just took out?
        sh_list = list(cleaned_demographics.index.values)    
        
#########################################################################################
    #now generate stats files as csvs and glm output (make sure metric and hemisphere is in file)
    #statstr = '(cd %s; setenv m %s ; setenv p %s ; ./run_generatestats.sh)' % (scripts_path, analysis_path, project)
    #stdin, stdout, stderr = client.exec_command(statstr)
    #time.sleep(30) 
    #glmstr = '(cd %s; setenv m %s ; setenv p %s ; ./run_groupglm.sh)' % (scripts_path, analysis_path, project)
    #stdin, stdout, stderr = client.exec_command(glmstr)
    #time.sleep(30) 

    # Now open each of the stats files and put a control in the first row for every file (how to know when job is done)

LDS0070120_20190620
LDS0110021_20181016
LDS0110022_20181129
LDS0110040_20190123
LDS0110041_20190212
LDS0110052_20190205
LDS0110053_20190227
LDS0110078_20190320
LDS0110079_20190411
LDS0180096_20190530
LDS0180117_20190719
LDS0180153_20190806
LDS0220026_20181109
LDS0220031_20181130
LDS0220050_20190208
LDS0220071_20190311
LDS0220081_20190322
LDS0220084_20190418
LDS0220088_20190408
LDS0220099_20190515
LDS0220123_20190701
LDS0220126_20190726
LDS0360098_20190524
LDS0360121_20190702
LDS0370001_20180509
LDS0370002_20180606
LDS0370005_20180802
LDS0370006_20180726
LDS0370007_20180801
LDS0370008_20180815
LDS0370010_20180815
LDS0370011_20180822
LDS0370012_20180824
LDS0370013_20180822
LDS0370014_20180913
LDS0370015_20181113
LDS0370016_20180912
LDS0370017_20181001
LDS0370018_20181015
LDS0370019_20181121
LDS0370020_20181212
LDS0370034_20190107
LDS0370037_20181218
LDS0370038_20181217
LDS0370042_20190108
LDS0370047_20190226
LDS0370058_20190318
LDS0370061_20190219
LDS0370065_20190227
LDS0370073_20190308


In [81]:
# RUN JOBS FOR ABOVE

for idx, pip in enumerate(pipelines): 
    analysis_path = '/autofs/cluster/animal/scan_data/leads/analyses_nip/'+pip+'/'

    #now generate stats files as csvs and glm output (make sure metric and hemisphere is in file)
    #statstr = '(cd %s; setenv m %s ; setenv p %s ; ./run_generatestats.sh)' % (scripts_path, analysis_path, project)
    #stdin, stdout, stderr = client.exec_command(statstr)
    #time.sleep(60) 
    glmstr = '(cd %s; setenv m %s ; setenv p %s ; ./run_groupglm.sh)' % (scripts_path, analysis_path, project)
    stdin, stdout, stderr = client.exec_command(glmstr)
    time.sleep(60) 


In [79]:
## SS GLMS (AMY NEG REPORT)
import shutil

analysis_path = '/autofs/cluster/animal/scan_data/leads/analyses_nip/RECON_FLAIR/'

# load in AMY - fsgd (generated above):
NEG_FSGD = pd.read_csv(analysis_path+'new.AMYNvsCN.fsgd')

AMY_NEG_DF = amyinfo[amyinfo.amyelg == 0]

# Make a FSGD file that has only controls and then add one N at a time in the next loop
total_negatives = []
with open(analysis_path+'new.AMYNvsCN.fsgd', 'r') as in_file:
    with open(analysis_path+'CN.fsgd', 'w') as result:
        for line in in_file:
            if line.endswith('\tN\n') is False:
                result.write(line)
            elif line.endswith('\tN\n'):
                total_negatives.append(line)
    result.close()
in_file.close()

total_negatives = total_negatives[5:]

# CREATE FSGFILE for INDIVIDUAL AMY- SUBJECT vs. all updated controls
for negative_amy_subject in total_negatives:
    # copy the CN file
    subname = negative_amy_subject.split('\t')[1].split('_')[0]
    print(subname)
    shutil.copy(analysis_path+'CN.fsgd',analysis_path+'CNvs'+subname+'.fsgd')
    # append one line
    with open(analysis_path+'CNvs'+subname+'.fsgd', 'a') as indiv_file:
        indiv_file.write(negative_amy_subject)
    indiv_file.close()
    
    #run 3 ssglm commands (per hemi)
    os.chdir(analysis_path) 
    glmstr = '(cd %s; setenv m %s ; setenv p %s ; setenv FSGD %s; ./run_singlesubjectglm.sh)' % (scripts_path, analysis_path, project, 'CNvs'+subname)
    stdin, stdout, stderr = client.exec_command(glmstr)
    time.sleep(60)
# move files to COMPARING_AMYNEG_CN folder

# can use other script to take screenshots

LDS0370014
LDS0370015
LDS0670048
LDS0670104
LDS0730024
LDS1770082
LDS3600068
LDS3600087
LDS3600093
LDS3600132
LDS0180117
LDS0220081
LDS0220084
LDS0220123
LDS0370011
LDS0370014
LDS0370015
LDS0670048
LDS0670104
LDS0730024
LDS1770082
LDS3600068
LDS3600087
LDS3600093
LDS3600132


In [80]:
total_neg = ['Input\tLDS0370029_20181210\tN\n']
# # CREATE FSGFILE for INDIVIDUAL AMY- SUBJECT vs. all updated controls
for negative_amy_subject in total_neg:
#     # copy the CN file
    subname = negative_amy_subject.split('\t')[1].split('_')[0]
    shutil.copy(analysis_path+'CN.fsgd',analysis_path+'CNvs'+subname+'.fsgd')
#     # append one line
    with open(analysis_path+'CNvs'+subname+'.fsgd', 'a') as indiv_file:
        indiv_file.write(negative_amy_subject)
    indiv_file.close()
    
#     #run 3 ssglm commands (per hemi)
    os.chdir(analysis_path) 
    glmstr = '(cd %s; setenv m %s ; setenv p %s ; setenv FSGD %s; ./run_singlesubjectglm.sh)' % (scripts_path, analysis_path, project, 'CNvs'+subname)
    stdin, stdout, stderr = client.exec_command(glmstr)
    time.sleep(1)


In [78]:
total_negatives[5:]

['Input\tLDS0370014_20180913\tN\n',
 'Input\tLDS0370015_20181113\tN\n',
 'Input\tLDS0670048_20190320\tN\n',
 'Input\tLDS0670104_20190614\tN\n',
 'Input\tLDS0730024_20181107\tN\n',
 'Input\tLDS1770082_20190129\tN\n',
 'Input\tLDS3600068_20190325\tN\n',
 'Input\tLDS3600087_20190417\tN\n',
 'Input\tLDS3600093_20190430\tN\n',
 'Input\tLDS3600132_20190731\tN\n',
 'Input\tLDS0180117_20190719\tN\n',
 'Input\tLDS0220081_20190322\tN\n',
 'Input\tLDS0220084_20190418\tN\n',
 'Input\tLDS0220123_20190701\tN\n',
 'Input\tLDS0370011_20180822\tN\n',
 'Input\tLDS0370014_20180913\tN\n',
 'Input\tLDS0370015_20181113\tN\n',
 'Input\tLDS0670048_20190320\tN\n',
 'Input\tLDS0670104_20190614\tN\n',
 'Input\tLDS0730024_20181107\tN\n',
 'Input\tLDS1770082_20190129\tN\n',
 'Input\tLDS3600068_20190325\tN\n',
 'Input\tLDS3600087_20190417\tN\n',
 'Input\tLDS3600093_20190430\tN\n',
 'Input\tLDS3600132_20190731\tN\n']